In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import pandas as pd
import cv2 as cv
import numpy as np
import pprint
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 10]
pd.options.display.max_columns = None
pd.options.display.width=None

from config import folder_locs, interest_cols, rando_seed

# Instructions:
1. Put all your photos in the input_photos folder
2. Put the annotation file in the input_csvs folder
3. Set the input_csv_filename variable (in the User Input section below) to the name of the annotation file
4. Run all the cells (top menu: Run > Run All Cells)

# User Input

enter the name of the csv file. 

if the file is an excel file, please save the file as a csv file by:
+ opening the excel file
+ selecting File from the menu at the top
+ selecting Save As
+ selecting CSV UTF-8 from the drop-down menu
+ selecting Save

In [3]:
input_csv_filename = 'your_annotation_filename_here.csv'

# Code

creating the relative filepath for the annoation file

In [37]:
input_csv_filepath = folder_locs['in_files'] + input_csv_filename
# print(input_csv_filepath)

loading the annotation file as a dataframe

In [36]:
csv_df = pd.read_csv(input_csv_filepath)
# csv_df

getting all the uniuqe labels in the annotation file

In [35]:
unique_labels = csv_df[interest_cols['label']].unique()
# print(unique_labels)

generating colors for the bounding boxes of the unique labels.  
a random seed has been set in the config.py file, under the scripts folder.  
this means that the colors generated for a certain annotation file will always be the same

In [27]:
np.random.seed(rando_seed)
rando_bgrs = np.random.choice(a=np.arange(start=30, 
                                          stop=225), 
                              size=3*len(unique_labels), 
                              replace=False).reshape(len(unique_labels), 3).tolist()

creating a dictionary with the unique labels as keys, and the bounding box colors as the values

In [34]:
label_bgrs = {}
for i, a_label in enumerate(unique_labels):
    label_bgrs[a_label] = tuple(rando_bgrs[i])
# pprint.pprint(label_bgrs)

iterate through each file to draw bounding boxes and labels.  
save the images in the output_photos folder.

If you want to see the step-by-step process of the code, uncomment all the 'print' blocks of code.  
please make sure you are working with a small dataset if you are doing this (the sample file contained 5 images). 

In [40]:
txt_color=(255, 255, 255) # the text color will be green
filenames = csv_df[interest_cols['filename']].unique() # get all the unique filenames 
n_files = filenames.shape[0] # get a count of the total number of unique filenames

for i, filename in enumerate(filenames):
    
    # print(f'\n\nWORKING WITH FILE {i+1} OF {n_files}')
    # print(f'\nFILENAME:\n{filename}')
    
    cur_df = csv_df[csv_df[interest_cols['filename']]==filename].copy() # subset the dataframe for one photo
    
    # print(f'\nTHESE ARE ALL THE ANNOTATIONS FOR THE FILE:\n')
    # for i, row in cur_df.iterrows():
    #     print(row)
    
    cur_filepath = folder_locs['in_pics'] + filename # create the filepath for the filename
    
    # print(f'\nTHIS IS THE FILEPATH USED TO READ THE IMAGE:\n{cur_filepath}')
    
    cur_img = cv.imread(cur_filepath) # load the image file
    
    # print(f'\nTHIS IS THE RAW IMAGE\n')
    # plt.imshow(cv.cvtColor(cur_img, cv.COLOR_BGR2RGB))
    # plt.show()
    
    
    line_thickness = max(round(sum(cur_img.shape) / 2 * 0.003), 2)  # calculate bbox line thickness
    
    # print(f'\nTHIS WILL BE THE LINE THICKNESS OF THE BBOX:\n{line_thickness}')
    
    # iterate through each row of the subset dataframe, and draw box
    
    # print(f'\nWE WILL NOW ITERATE THROUGH EACH ANNOTATION')
    
    n_rows = cur_df.shape[0]
    cur_row = 0
    for i, row in cur_df.iterrows():
        
        # print(f'\n\tROW {cur_row+1} of {n_rows}')
        
        cur_label = row[interest_cols['label']] # get the label of the row
        
        # print(f'\n\tTHIS IS THE CURRENT ANNOTATION LABEL:\n\t{cur_label}')
        
        # get the color (BGR) for the label
        cur_col = label_bgrs[cur_label]
        
        # print(f'\n\tTHIS IS THE BGR FOR THE LABEL:\n\t{cur_col}')
        
        # make rectangle xy coordinates
        x1 = row[interest_cols['x1']]
        y1 = row[interest_cols['y1']]
        x2 = row[interest_cols['x2']]
        y2 = row[interest_cols['y2']]
        coord_1, coord_2 = (x1, y1), (x2, y2)
        
        # print(f'\n\tTHIS IS THE TOP LEFT COORDINATE:\n\t{coord_1}')
        # print(f'\n\tTHIS IS THE BOTTOM RIGHT COORDINATE:\n\t{coord_2}')
        
        # draw the bounding box
        cv.rectangle(cur_img, 
                     coord_1, 
                     coord_2, 
                     cur_col, 
                     thickness=line_thickness, 
                     lineType=cv.LINE_AA,
                    )
        
        # print(f'\n\tTHIS IS THE IMAGE WITH ADDED BBOX:\n\t')
        # plt.imshow(cv.cvtColor(cur_img, cv.COLOR_BGR2RGB))
        # plt.show()
        
        # insert text label
        font_thickness = max(line_thickness - 1, 1)
        w, h = cv.getTextSize(cur_label, 
                              0, 
                              fontScale=line_thickness / 3, 
                              thickness=font_thickness)[0]
        coord_2 = coord_1[0] + w, coord_1[1] - h - 3
        cv.rectangle(cur_img, 
                     coord_1, 
                     coord_2, 
                     cur_col, 
                     -1, 
                     cv.LINE_AA,
                    )
        cv.putText(cur_img, 
                   cur_label, 
                   (coord_1[0], coord_1[1] - 2), 
                   0, 
                   line_thickness / 3, 
                   txt_color, 
                   thickness=font_thickness, 
                   lineType=cv.LINE_AA,
                  )
        
        # print(f'\n\tTHIS IS THE IMAGE WITH ADDED LABEL TEXT:\n\t')
        # plt.imshow(cv.cvtColor(cur_img, cv.COLOR_BGR2RGB))
        # plt.show()
        
        cur_row += 1
    
    
    out_filepath = folder_locs['out_pics'] + filename # create relative filepath of the image to be saved
    
    # print(f'\nSAVING THE FILE TO HERE:\n{out_filepath}')
    
    cv.imwrite(out_filepath, cur_img) # save the file
    